## Capturing : Capture the geometry from the sampling and convert to file array ##
Maintain: Demo and test only. <br/>
For real calculation, a separate UNIX shell script that run the calculation and another separate Python script to capture the results (either directly from the .xyz file or summary from csv file) is required


In [8]:
import os
import sys
import pandas as pd

sys.path.insert(0, "..")

In [9]:
from ase.calculators.nwchem import NWChem
os.environ['ASE_NWCHEM_COMMAND'] = 'mpirun -n 8 nwchem PREFIX.nw > PREFIX.out'
nwchem_calc = NWChem(label='calc/nwchem',
              maxiter=2000,
              xc='b3lyp',
              basis='6-31G**')



In [10]:
import numpy as np
from ase.units import Hartree, mol, kcal, Bohr
def nwchem_compute(md_samples_arr, nwchem_calc):
    """
    Take the array with the atoms, compute the DFT energy for all the given configurations.
    """
    
    md_dft_energy_arr = np.zeros(md_samples_arr.size)
    md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
    for index, atoms in md_samplesArr.items():
        atoms.set_calculator(nwchem_calc)
        potent_e = atoms.get_potential_energy()
        force_arr = atoms.get_forces()
        print(index)
        print(potent_e)
        md_dft_energy_arr[index] = potent_e
        md_dft_force_arr[index] = force_arr
    
    return md_dft_energy_arr, md_dft_force_arr

def get_energy_force_mol(atoms, calc):
    """Get the energy for each molecule for a particular molecule (atoms)
    
            Args:
                atoms: an ase.Atoms object
                calc: an ase.calculator
                
            Outputs:
                poten_e*: potential energy for the molecule
                force_arr*: force for the molecule
                
                * Both wrapped in a Pandas.Series() object
    
    Comments:
    1. For FileIO Calculator, May improve the performance by 
    getting all the character at once.
    
    """
    atoms.set_calculator(calc)
    poten_e = atoms.get_potential_energy()
    force_arr = atoms.get_forces()
    
    return pd.Series([poten_e, force_arr])
    
# def dft_compute(md_samples_arr, dft_calc):
#     """Compute the DFT Energy for the given array using the given calculator
    
#             Args:
#                 md_samplesArr: a pd.Series() of all the configurations 
#                 dft_calc:  a ase.calculator that will be assigned to the atoms
                
#             Outputs:
#                 md_dft_energy_arr: an numpy array of all the dft energy
    
#     Comments: 
#     1. Attempt to generalize in the naming.
#     2. Attempt to use element-wise operations in pandas.Series(), 
#     which may not work for smaller sets.
#     3. Untested. 
#     """
    
#     md_dft_energy_arr = np.zeros(md_samples_arr.size)
#     md_samples_arr.apply(

In [11]:
## Global Var:
relative_energy_ref_dft = 0
relative_energy_ref_dftb = 0

def get_relative_energy(energy_arr):
    """
    Get the relative energy with reference to the lowest point in all the energy. 
        Args:
            energy_arr: an np.array of energy (either dftb or dft) of shape (No. samples, )
    
    """
    lowest_energy = np.amin(energy_arr)
    relative_energy_arr = energy_arr - lowest_energy
    
    return relative_energy_arr, lowest_energy


In [12]:
from dftb_traj_io import read_dftb_atom_from_traj
from dftb_traj_io import read_dftb_md_energy

In [13]:
##########
# WARNING: TEMPORARY!
# Use it only because the original file has been lost. 
##########
# md_samplesArr, md_Mulliken = read_dftb_atom_from_traj('../md_sampling/geom.out.xyz', writeTrajectory=True)
# md_Mulliken.shape
md_samplesArr = pd.read_pickle("/home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_opt/md_samples_arr.pkl")

In [14]:
md_samplesArr

0       (Atom('C', [-0.62822424, 0.58239182, 2.5289634...
1       (Atom('C', [-0.6279525, 0.58270682, 2.5290658]...
2       (Atom('C', [-0.62768124, 0.58302167, 2.5291682...
3       (Atom('C', [-0.62741095, 0.5833362, 2.52927095...
4       (Atom('C', [-0.62714212, 0.58365023, 2.5293739...
5       (Atom('C', [-0.62687526, 0.58396356, 2.5294774...
6       (Atom('C', [-0.62661086, 0.58427602, 2.5295815...
7       (Atom('C', [-0.62634942, 0.58458743, 2.5296862...
8       (Atom('C', [-0.62609143, 0.58489762, 2.5297917...
9       (Atom('C', [-0.62583741, 0.58520639, 2.5298982...
10      (Atom('C', [-0.62558782, 0.58551358, 2.5300057...
11      (Atom('C', [-0.62534318, 0.585819, 2.53011434]...
12      (Atom('C', [-0.62510397, 0.58612249, 2.5302242...
13      (Atom('C', [-0.62487067, 0.58642388, 2.5303354...
14      (Atom('C', [-0.62464375, 0.58672299, 2.5304481...
15      (Atom('C', [-0.6244237, 0.58701965, 2.53056237...
16      (Atom('C', [-0.62421098, 0.58731372, 2.5306783...
17      (Atom(

In [15]:
sys.path.insert(0, "../Calculator")

In [16]:
from ASEDFTBMod import DftbMod
os.environ['DFTB_PREFIX'] = '/home/junmian/Applications/dftbplus/mio-1-1/'
os.environ['DFTB_COMMAND'] = '/home/junmian/Applications/dftbplus/_install/bin/dftb+'


    # Set path to the model


dftb_calc = DftbMod(
                         Hamiltonian_MaxAngularMomentum_='',
                         Hamiltonian_MaxAngularMomentum_O='"p"',
                         Hamiltonian_MaxAngularMomentum_H='"s"',
                         Hamiltonian_MaxAngularMomentum_C='"p"',
                         Hamiltonian_MaxAngularMomentum_N='"p"',
                         )


In [17]:
#md_energy_arr, md_kinetic_energy_arr = read_dftb_md_energy('../md_sampling/md.out')
md_energy_arr, dft_force_arr = nwchem_compute(md_samplesArr, dftb_calc)
md_energy_arr.shape
print(md_energy_arr)

0
-391.55404580086423
1
-391.5498391976077
2
-391.54488020584876
3
-391.5391849216502
4
-391.53277863663743
5
-391.525695820978
6
-391.5179798748488
7
-391.50968279094997
8
-391.5008642193145
9
-391.4915909196249
10
-391.48193556644276
11
-391.47197643937454
12
-391.46179589677666
13
-391.45147931434826
14
-391.44111387457286
15
-391.43078728621305
16
-391.4205863018125
17
-391.410595491186
18
-391.4008959015665
19
-391.391564047279
20
-391.3826705963992
21
-391.374279550036
22
-391.3664472509536
23
-391.35922136396533
24
-391.35264055605614
25
-391.3467339192699
26
-391.3415206355385
27
-391.3370101716432
28
-391.33320223381804
29
-391.3300869060572
30
-391.3276455191755
31
-391.3258510355127
32
-391.32466882213095
33
-391.32405763174467
34
-391.32397058628686
35
-391.32435627803966
36
-391.32515966347677
37
-391.326323538153
38
-391.32778918922827
39
-391.32949782198864
40
-391.3313911761708
41
-391.33341281602145
42
-391.33550858713267
43
-391.3376280525855
44
-391.3397244966452
45


359
-391.49015045460857
360
-391.48543186699595
361
-391.4807860030507
362
-391.47622888934575
363
-391.47177457009093
364
-391.4674352653348
365
-391.4632214757308
366
-391.4591419342344
367
-391.45520365943497
368
-391.4514120121415
369
-391.44777065953446
370
-391.4442814729328
371
-391.4409449057162
372
-391.4377594153775
373
-391.43472217150514
374
-391.43182846036876
375
-391.4290722997928
376
-391.4264461447074
377
-391.4239409879671
378
-391.4215465500354
379
-391.41925122821664
380
-391.41704236903144
381
-391.41490611372353
382
-391.41282786469856
383
-391.41079246651805
384
-391.40878432198286
385
-391.40678798871033
386
-391.4047881903481
387
-391.40277008518575
388
-391.40071955967596
389
-391.39862337318027
390
-391.3964695344706
391
-391.39424760649746
392
-391.39194888205003
393
-391.3895664937718
394
-391.38709582693537
395
-391.38453437034616
396
-391.3818818572457
397
-391.37914018808783
398
-391.3763135676768
399
-391.3734084088382
400
-391.3704335344504
401
-391.36

711
-391.4994146802669
712
-391.4994629971169
713
-391.4994380499467
714
-391.4993282836809
715
-391.4991248150481
716
-391.49882152719545
717
-391.4984149172148
718
-391.4979041094465
719
-391.49729055860575
720
-391.4965775151036
721
-391.49577000095366
722
-391.49487396292676
723
-391.49389649451973
724
-391.49284497053515
725
-391.49172737466847
726
-391.4905511606708
727
-391.48932320803766
728
-391.488049173842
729
-391.4867330065431
730
-391.4853773719124
731
-391.48398228608914
732
-391.4825461025013
733
-391.4810648495303
734
-391.479532738223
735
-391.4779420548753
736
-391.47628322021677
737
-391.474545519511
738
-391.4727164913882
739
-391.4707831459871
740
-391.4687320039164
741
-391.46654966496703
742
-391.46422335280283
743
-391.46174150015474
744
-391.45909342150327
745
-391.45627024941433
746
-391.45326532438
747
-391.4500749596058
748
-391.4466985028266
749
-391.4431387390299
750
-391.4394019208697
751
-391.43549745449764
752
-391.4314383242374
753
-391.4272409238662


In [18]:
md_dft_energy_arr, md_dft_force_arr = nwchem_compute(md_samplesArr, nwchem_calc)

0
-7739.83382816373
1
-7739.836456887459
2
-7739.838314557244
3
-7739.839402379013
4
-7739.839730694018
5
-7739.839318759078
6
-7739.838194604012
7
-7739.83639452288
8
-7739.8339627496125
9
-7739.830950724151
10
-7739.827416240214
11
-7739.8234227516505
12
-7739.819038222899
13
-7739.8143341499435
14
-7739.809384479833
15
-7739.8042642303335
16
-7739.799048426447
17
-7739.793810739408
18
-7739.788622437716
19
-7739.783550981117
20
-7739.778659224836
21
-7739.774004068129
22
-7739.769635959245
23
-7739.7655977319255
24
-7739.761924302405
25
-7739.758642186114
26
-7739.755769083295
27
-7739.753314028128
28
-7739.751277388376
29
-7739.749651085823
30
-7739.748419130621
31
-7739.747558247566
32
-7739.747038599046
33
-7739.746824388484
34
-7739.746875139407
35
-7739.747146362721
36
-7739.7475907423695
37
-7739.748159168577
38
-7739.748801622053
39
-7739.74946829595
40
-7739.75011043968
41
-7739.7506812393995
42
-7739.751136562923
43
-7739.751435776031
44
-7739.751542099761
45
-7739.75142337

361
-7739.6754676600785
362
-7739.675031198971
363
-7739.674841707096
364
-7739.674906126107
365
-7739.67522926333
366
-7739.675814242931
367
-7739.676661769766
368
-7739.677771014161
369
-7739.679138968061
370
-7739.680760526283
371
-7739.682628949437
372
-7739.684735206581
373
-7739.687068366276
374
-7739.689615388416
375
-7739.692361410199
376
-7739.695289543475
377
-7739.6983809549165
378
-7739.701615025642
379
-7739.704969524166
380
-7739.708420585178
381
-7739.711942999343
382
-7739.715510389029
383
-7739.7190954765665
384
-7739.722670361357
385
-7739.726206969715
386
-7739.729677009742
387
-7739.733052767353
388
-7739.736307234454
389
-7739.7394143659685
390
-7739.742349527456
391
-7739.745089712097
392
-7739.747614012513
393
-7739.749903617281
394
-7739.75194216934
395
-7739.753715916709
396
-7739.75521376141
397
-7739.756427374343
398
-7739.757351264954
399
-7739.757982686309
400
-7739.758321598162
401
-7739.7583705933785
402
-7739.758134781249
403
-7739.757621551243
404
-7739

718
-7739.67091286699
719
-7739.671776192593
720
-7739.672622608988
721
-7739.673457553387
722
-7739.674287538348
723
-7739.675119519782
724
-7739.675960588948
725
-7739.676817592175
726
-7739.677696846209
727
-7739.678603667036
728
-7739.67954197903
729
-7739.680514151564
730
-7739.681520819891
731
-7739.682560642899
732
-7739.683630179297
733
-7739.684723954209
734
-7739.685834480181
735
-7739.68695231219
736
-7739.688066299558
737
-7739.689163965727
738
-7739.690231583205
739
-7739.691254760872
740
-7739.69221865592
741
-7739.693108545524
742
-7739.693910279733
743
-7739.694610625135
744
-7739.695197892777
745
-7739.695662185692
746
-7739.695995902673
747
-7739.696194111443
748
-7739.6962548166985
749
-7739.696179044962
750
-7739.695971241513
751
-7739.695639077238
752
-7739.695193538657
753
-7739.694648719197
754
-7739.694021589649
755
-7739.693331662671
756
-7739.692600531749
757
-7739.691851268304
758
-7739.6911080578675
759
-7739.690395435179
760
-7739.689737541035
761
-7739.689

In [19]:
md_dft_rel_e_arr, dft_ref = get_relative_energy(md_dft_energy_arr)
print(dft_ref)
print(md_dft_rel_e_arr)

-7739.839730694018
[0.00590253 0.00327381 0.00141614 ... 0.07443298 0.07281333 0.07112041]


In [20]:
#md_dft_rel_energy_arr, relative_energy_ref_dft= get_relative_energy(md_dft_energy_arr)
md_dftb_rel_e_arr, relative_energy_ref_dftb = get_relative_energy(md_energy_arr)
#print(relative_energy_ref_dft)
print(relative_energy_ref_dftb)
print(md_dftb_rel_e_arr)
#print(md_dft_rel_energy_arr)

-391.6073901359587
[0.05334434 0.05755094 0.06250993 ... 0.31444587 0.30828034 0.30175506]


In [21]:
#np.savetxt('md_energy.txt',md_energy_arr)

In [22]:
#np.savetxt('md_mulliken.txt', md_Mulliken)

In [23]:
#md_samplesArr.to_pickle("./md_samples_arr.pkl")

In [24]:
#np.savetxt('nwchem_pbe321G.txt', md_dft_e_arr)

In [25]:
#np.savetxt('md_dft_rel_energy.txt', md_dft_rel_energy_arr)
#np.savetxt('md_dftb_rel_energy_arr', md_dftb_rel_e_arr)

In [26]:
#np.savetxt('dft_e_arr.txt', dft_e_arr)

## Save the Bias ##

In [27]:
import pandas as pd
relative_energy = pd.Series([])
relative_energy['dft'] = dft_ref #relative_energy_ref_dft
relative_energy['calc'] = relative_energy_ref_dftb
relative_energy.to_pickle('../relative_energy.pkl')

In [28]:
# import pandas as pd
# relative_energy = pd.Series([])
# relative_energy['dft'] = -7685.457972419566
# relative_energy['calc'] = -389.0714894952025
# relative_energy.to_pickle('../model/dftb/relative_energy.pkl')

In [29]:
import WriteTrain
#import importlib
#importlib.reload(ZipTrain)

In [34]:
WriteTrain.write_train_samples("glycine_opt_b3lyp631G_real1", md_samplesArr, md_dftb_rel_e_arr, md_dft_rel_e_arr, 
                  relative_energy, 
                  md_energy_arr, md_dft_energy_arr, 
                  np.zeros( (1,1) ,dtype=np.float32), 
                    dft_force_arr, md_dft_force_arr)


md_samples_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_opt_b3lyp631G_real1/md_samples_arr.pkl
md_calc_rel_energy_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_opt_b3lyp631G_real1/md_calc_rel_e_arr.txt
rel_e_ref saved to /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_opt_b3lyp631G_real1/rel_e_ref.pkl
md_high_rel_energy_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_opt_b3lyp631G_real1/md_high_rel_e_arr.txt


In [ ]:
WriteTrain.write_train_samples

In [ ]:
import os
import sys 

from utils.DirNav import get_project_dir

import matplotlib.pyplot as plt
from ase.io.trajectory import Trajectory



from ase.md.verlet import VelocityVerlet
from ase import units
from ase.io import read, write

project_dir = get_project_dir() 
glycine_test = read(os.path.join(project_dir, 'md_sampling','example_molecules' , 'glycine_opt.xyz'))
print(glycine_test.get_chemical_symbols())

glycine_test.set_calculator(nwchem_calc)

In [ ]:
dyn = VelocityVerlet(glycine_test, 0.1 * units.fs)  # 5 fs time step.
traj = Trajectory('glycine_test_b3lyp631G_real.traj', 'w', glycine_test)
dyn.attach(traj.write, interval=1)

printenergy(glycine_test)
dyn.run(2000)